# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 32
REG_LAMBDA = 5e-4
NUM_GPUS = 2

In [ ]:
#dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(input_seq_length=1, target_seq_length=0)
#dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(input_seq_length=1, target_seq_length=0)
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(input_seq_length=1, target_seq_length=0)

In [ ]:
dataset_train = tt.datasets.mnist.MNISTTrainDataset()
dataset_valid = tt.datasets.mnist.MNISTValidDataset()
#dataset_test = tt.datasets.mnist.MNISTTestDataset()

In [ ]:
dataset_train = tt.datasets.ucf11.UCF11TrainDataset(input_seq_length=1, target_seq_length=1,
                                                    image_size=(60, 80, 1),
                                                    min_exampled_in_queue=256,
                                                    num_threads=8,
                                                    do_distortion=False)
dataset_valid = tt.datasets.ucf11.UCF11TrainDataset(input_seq_length=1, target_seq_length=1,
                                                    image_size=(60, 80, 1),
                                                    min_exampled_in_queue=256,
                                                    num_threads=8,
                                                    do_distortion=False)

In [ ]:
class SimpleAutoencoderModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleAutoencoderModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, is_training=True,
                  device_scope=None, memory_device=None):
        input_shape = inputs.get_shape()
        
        x = tf.contrib.layers.flatten(inputs)
        encoded = tt.network.fc("FC_Enc", x, 64,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.relu,
                                device=memory_device)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, x.get_shape()[1],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.sigmoid,
                                device=memory_device)

        return tf.reshape(decoded, [-1] + [int(x) for x in input_shape[1:]])
    
    @tt.utils.attr.override
    def loss(self, predictions, targets):
        return tt.loss.mse(predictions, targets)

In [ ]:
#runtime = tt.core.MultiGpuRuntime(NUM_GPUS)
runtime = tt.core.DefaultRuntime()
runtime.register_datasets(dataset_train, dataset_valid)
runtime.register_model(SimpleAutoencoderModel(reg_lambda=REG_LAMBDA))
runtime.build(is_autoencoder=True)

In [ ]:
runtime.train(BATCH_SIZE, steps=1000)

In [ ]:
runtime.validate(batch_size=50)

In [ ]:
runtime.test(batch_size=50)

In [ ]:
# because we use a queue, we have to launch all these ops in the same run.
inputs, inputs2, targets = runtime.run([runtime._x, runtime._x, runtime._y_,])

# this prediction uses another input compared to the run above.
predictions = runtime.predict(runtime._x)

print(inputs.min(), inputs.max())
print(targets.min(), targets.max())
print(predictions.min(), predictions.max())

tt.visualization.display_batch((inputs + 1) * 127.5, title="Inputs")
tt.visualization.display_batch((inputs2 + 1) * 127.5, title="Inputs2")
tt.visualization.display_batch((targets + 1) * 127.5, title="Targets")
tt.visualization.display_batch(predictions * 255, title="Predictions")

In [ ]:
runtime.close()